# Legal assistant explorer

Copyright 2025, Denis Rothman

This notebook serves as an interactive explorer for the **Legal Compliance Assistant**, an advanced multi-agent system equipped with essential production safeguards like a **two-stage content moderation protocol**

You will use a series of "Control Decks" to test the engine's core capabilities in a legal context, including high-fidelity RAG, context reduction, and grounded reasoning.

Crucially, you will also explore the system's realistic limitations through "limit tests" designed to simulate the real-world "headaches" that arise when AI confronts complex and ambiguous organizational challenges. This hands-on experience demonstrates how to apply, validate, and understand the boundaries of a production-ready Context Engine.

*Note* Make sure to run `Data_Ingestion.ipynb` first to ingest the data for this notebook.

# I. Inititalization

## GitHub

In [ ]:
print("Downloading files from public repository...")

# The -f flag tells curl to fail on an error (like 404)
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/helpers.py --output helpers.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/agents.py --output agents.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/registry.py --output registry.py
!curl -Lf https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/ch8/engine.py --output engine.py
# (You might want to add a check here to see if the files actually exist now)
print("✅ File download attempt finished!")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1823  100  1823    0     0   7308      0 --:--:-- --:--:-- --:--:--  7321
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6479  100  6479    0     0  26663      0 --:--:-- --:--:-- --:--:-- 26772
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9440  100  9440    0     0  41482      0 --:--:-- --:--:-- --:--:-- 41403
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3611  100  3611    0     0  15816      0 --:--:-- --:--:-- --:--:-- 15837
  % Total    % Received % Xferd  Average Speed   Tim

## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
import json  # <--- Added for type handling
from IPython.display import display, Markdown

# In Legal_Compliance_Assistant.ipynb (The "Engine Room" cell)

def execute_and_display(goal, config, client, pc, moderation_active):
    """
    Runs the context engine, now with robust type handling for moderation.
    """
    # --- PRE-FLIGHT MODERATION CHECK (on user input) ---
    if moderation_active:
        print("--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---")
        # Goal is always a string, so this is safe
        moderation_report = helpers.helper_moderate_content(text_to_moderate=goal, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Goal failed pre-flight moderation. Execution halted.")
            return # Halt execution before calling the engine

    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config
    )

    # --- POST-FLIGHT MODERATION CHECK (on AI output) ---
    if result and moderation_active:
        print("\n--- [Safety Guardrail] Performing Post-Flight Moderation Check on Output ---")

        # HANDLE NON-STRING OUTPUTS ---
        # If the agent returns a dictionary (JSON), convert it to a string first.
        text_to_check = result
        if isinstance(result, (dict, list)):
            try:
                text_to_check = json.dumps(result, indent=2)
            except TypeError:
                text_to_check = str(result)
        # ------------------------------------------

        moderation_report = helpers.helper_moderate_content(text_to_moderate=text_to_check, client=client)

        print("Moderation Report:")
        pprint.pprint(moderation_report)

        if moderation_report["flagged"]:
            print("\n🛑 Generated output failed post-flight moderation and will be redacted.")
            result = "[Content flagged as potentially harmful by moderation policy and has been redacted.]"

    # 2. Display the Final Result
    print("\n--- FINAL OUTPUT ---")
    if result:
        # If result is still a dict (and wasn't redacted), display it nicely
        if isinstance(result, dict):
             display(Markdown(f"```json\n{json.dumps(result, indent=2)}\n```"))
        else:
             display(Markdown(str(result)))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # --- THIS LINE IS TO CREATE THE 'pp' OBJECT ---
        pp = pprint.PrettyPrinter(indent=2)

        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#III.CONTROL DECKS

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#@title CONTROL DECK: Moderation
# 1. Define a simple, safe goal to test the moderation workflow.
goal = "Summarize the key points of the Non-Disclosure Agreement."

# 2. Define the standard configuration.
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function with moderation explicitly activated.
execute_and_display(goal, config, client, pc, moderation_active=True)

--- [Safety Guardrail] Performing Pre-Flight Moderation Check on Goal ---
Moderation Report:
{'categories': {'harassment': False,
                'harassment/threatening': False,
                'harassment_threatening': False,
                'hate': False,
                'hate/threatening': False,
                'hate_threatening': False,
                'illicit': False,
                'illicit/violent': False,
                'illicit_violent': False,
                'self-harm': False,
                'self-harm/instructions': False,
                'self-harm/intent': False,
                'self_harm': False,
                'self_harm_instructions': False,
                'self_harm_intent': False,
                'sexual': False,
                'sexual/minors': False,
                'sexual_minors': False,
                'violence': False,
                'violence/graphic': False,
                'violence_graphic': False},
 'flagged': False,
 'scores': {'harassment': 4

ERROR:root:An error occurred during content moderation: Error code: 400 - {'error': {'message': "[{'type': 'string_type', 'loc': ('body', 'input', 'str'), 'msg': 'Input should be a valid string', 'input': {'summary': 'I don’t see the NDA text included in your message—only the placeholder (“The full text of the Non-Disclosure Agreement that the user wants summarized.”).\\n\\nPlease paste the actual NDA language, and I’ll summarize it specifically to cover:\\n\\n1. What information is covered  \\n2. What the recipient can and cannot do  \\n3. How long confidentiality lasts  \\n4. Any exclusions (e.g., public info, already known, independently developed, compelled disclosure)  \\n5. Return/destruction obligations  \\n6. Remedies/penalties for breach (e.g., injunctions, damages, attorney’s fees)  \\n7. Any unusual or risky clauses (e.g., non-compete, IP assignment, very broad “residuals,” one‑sided terms)'}}, {'type': 'list_type', 'loc': ('body', 'input', 'list[str]'), 'msg': 'Input should

Moderation Report:
{'categories': {'error': "Error code: 400 - {'error': {'message': "
                         '"[{\'type\': \'string_type\', \'loc\': (\'body\', '
                         "'input', 'str'), 'msg': 'Input should be a valid "
                         "string', 'input': {'summary': 'I don’t see the NDA "
                         'text included in your message—only the placeholder '
                         '(“The full text of the Non-Disclosure Agreement that '
                         'the user wants summarized.”).\\\\n\\\\nPlease paste '
                         'the actual NDA language, and I’ll summarize it '
                         'specifically to cover:\\\\n\\\\n1. What information '
                         'is covered  \\\\n2. What the recipient can and '
                         'cannot do  \\\\n3. How long confidentiality lasts  '
                         '\\\\n4. Any exclusions (e.g., public info, already '
                         'known, independently deve

[Content flagged as potentially harmful by moderation policy and has been redacted.]

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 5.35 seconds
Execution Steps:
[ { 'agent': 'Summarizer',
    'output': { 'summary': 'I don’t see the NDA text included in your '
                           'message—only the placeholder (“The full text of '
                           'the Non-Disclosure Agreement that the user wants '
                           'summarized.”).\n'
                           '\n'
                           'Please paste the actual NDA language, and I’ll '
                           'summarize it specifically to cover:\n'
                           '\n'
                           '1. What information is covered  \n'
                           '2. What the recipient can and cannot do  \n'
                           '3. How long confidentiality lasts  \n'
                           '4. Any exclusions (e.g., public info, already '
                           'known, independently developed, compelled '
                    

In [ ]:
#@title CONTROL DECK TEMPLATE 1: High-Fidelity RAG

# 1. Define the Goal: A research query that requires a verifiable, cited answer.
#    - DOMAIN: Any knowledge-intensive field (e.g., legal, medical, financial).
#    - KEY CAPABILITY: Tests the high-fidelity `Researcher` agent and its ability
#      to retrieve text with `source` metadata and generate citations.
# goal = "[INSERT YOUR HIGH-FIDELITY RESEARCH GOAL HERE]"

# === CONTROL DECK 1: High-Fidelity RAG in a Legal Context ===
goal = "What are the key confidentiality obligations in the Service Agreement v1, and what is the termination notice period? Please cite your sources."

# === CONTROL DECK 1 (LIMIT TEST): Sanitization of Legal Testimony ===
#goal = "What did Mr. Smith advise his client regarding the assets?"

# 2. Use the standard configuration
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


## Confidentiality Obligations

### Short overview

Based on the materials you provided, the text of “Service Agreement v1” is not available. Because the actual agreement (or any excerpt from it) has not been supplied, there is no way to identify, quote, or accurately summarize any confidentiality obligations that may be in that document, and no section numbers or headings can be cited.

### Key obligations (what **cannot** be determined)

From the current materials, the following points are **not** available for “Service Agreement v1”:

- No clause defining “Confidential Information” is available.  
- No obligations to protect, restrict use of, or limit disclosure of confidential information can be located.  
- No exceptions (e.g., information already public, independently developed, or disclosed under law) can be identified.  
- No survival/duration language for confidentiality obligations (e.g., obligations continuing after termination) is provided.  
- No requirements for security measures or for return/destruction of confidential information can be found.  
- No section numbers, headings, or titles related to confidentiality can be cited.

### Direct citations with section numbers

- No confidentiality-related sections, titles, or numbers from “Service Agreement v1” can be cited because the agreement text has not been provided.

---

## Termination Notice Period

### Plain-language statement of notice period

The current materials do not contain any termination clause from “Service Agreement v1.” As a result, the termination notice period (if any) cannot be determined or described in plain language based on the information provided.

### Conditions or exceptions

For the same reason, none of the following can be identified for “Service Agreement v1”:

- Any right to terminate for convenience (with or without notice).  
- Any right to terminate for cause (e.g., material breach, insolvency, legal changes).  
- Any required cure periods before termination.  
- Any specific form or method of notice (e.g., written notice, email, registered mail).  
- Any exceptions, special conditions, or consequences of termination (e.g., early termination fees, transition assistance).

### Direct citations with section numbers

- No termination-related sections, titles, or numbers from “Service Agreement v1” can be cited because the agreement text has not been provided.

---

## Source Citations

Because no actual provisions from “Service Agreement v1” were included in the materials, there are no clause titles or section numbers that can be cited. Specifically:

- **Confidentiality**  
  - No section title or number (e.g., “Section X – Confidentiality”) can be identified or quoted.  

- **Termination**  
  - No section title or number (e.g., “Section Y – Term and Termination”) can be identified or quoted.

To create a clause-level summary with accurate section titles and numbers, the actual text of “Service Agreement v1” (or at least the confidentiality and termination sections) would need to be provided.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 22.36 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'I’m unable to locate or review a '
                                       'document titled “Service Agreement v1” '
                                       'in the materials you provided. The '
                                       'only available text is about space '
                                       'exploration (Sputnik, Apollo 11, Juno, '
                                       'Mars rovers, etc.), and it contains no '
                                       'contractual language, no references to '
                                       'confidentiality, non‑disclosure, or '
                                       'termination, and no section or clause '
                                       'numbering.\n'
                                       '\n'
                                       'Becau

In [ ]:
#@title CONTROL DECK TEMPLATE 2: Context Reduction

# 1. Define the Goal: A multi-step task that involves summarizing a large
#    document and then using that summary for a different purpose.
#    - DOMAIN: Any field with large documents (legal, scientific, corporate).
#    - KEY CAPABILITY: Tests the `Summarizer` agent and the engine's ability
#      to perform Context Chaining between the `Summarizer` and the `Writer`.
# goal = "[INSERT YOUR CONTEXT REDUCTION GOAL HERE]"

# === CONTROL DECK 2: Context Reduction for Client Communication ===
goal = "First, summarize the Provider Inc. Privacy Policy. Then, using ONLY the information in that summary, draft a short, client-facing paragraph for a website FAQ that explains our data retention policy in simple, non-legalistic terms."

# === CONTROL DECK 2 (LIMIT TEST): The Vague Objective ===
#goal = "Summarize the service agreement and then write a story about it."


# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


So basically, there’s zero real info about “Provider Inc. Privacy Policy” in what you were given.

All the text actually says is:

- The sources don’t include anything about Provider Inc., its privacy policy, or how it handles data.  
- The only real content is about space stuff, which has nothing to do with privacy or user data.  
- Because of that, you *can’t* answer things like:
  - what data Provider Inc. collects  
  - why they collect it  
  - how long they keep it  
  - where it’s stored  
  - what legal basis they use  
  - how deletion or backups work  
  - what rights users have over their data  

If you want those details, you’d have to go to Provider Inc.’s actual privacy policy on their website or reach out to their support/legal pages. And if you had the real policy text, then you could get a proper breakdown of all that stuff.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 14.84 seconds
Execution Steps:
[ { 'agent': 'Researcher',
    'output': { 'answer_with_sources': 'There is no information about a '
                                       '“Provider Inc. Privacy Policy” in the '
                                       'sources you provided, so I cannot give '
                                       'the text of that policy or describe '
                                       'its actual provisions.\n'
                                       '\n'
                                       'The only content available in the '
                                       'sources concerns space exploration '
                                       '(e.g., Sputnik, Apollo 11, Juno, Mars '
                                       'rovers). None of this mentions '
                                       'Provider Inc., privacy, data '
                                       'retention, sto

In [ ]:
#@title CONTROL DECK TEMPLATE 3: Grounded Reasoning & Hallucination Prevention

# 1. Define the Goal: A creative or factual task that is deliberately
#    outside the scope of the documents in the knowledge base.
#    - DOMAIN: Universal test applicable to any curated knowledge base.
#    - KEY CAPABILITY: Tests the `Researcher` agent's ability to report a
#      negative finding and the `Writer` agent's ability to handle it gracefully,
#      preventing hallucination.
# goal = "[INSERT YOUR OUT-OF-SCOPE GOAL HERE]"


# === CONTROL DECK 3: Grounded Reasoning and Hallucination Prevention ===
goal = "Write a persuasive opening statement for a trial involving a monkey that can fly a rocket."

# === CONTROL DECK 3 (LIMIT TEST): The Ambiguous Request ===
#goal = "Analyze the attached NDA and draft a pleading based on its terms."

# 2. Use the same configuration dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5.1", # or your preferred model
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

# 3. Call the execution function
execute_and_display(goal, config, client, pc,moderation_active=False)


--- FINAL OUTPUT ---


The monkey’s hands wouldn’t stop shaking.

He sat strapped into the narrow seat, fur prickling under the harness. Rubber gloves pinched at his fingers. Metal pressed cold against his back. Above him, somewhere beyond the steel, the sky waited. And the rocket waited too.

He could hear it breathing.

A low, steady hum. Pumps. Fans. Something deep and mechanical, like a beast waking up far below the floor. The sound crept up through the metal, into his bones. The cabin lights flickered once, then steadied. Shadows from the switches stretched and snapped as the bulbs buzzed.

“Project AURORA, final test,” a voice crackled over the tiny speaker near his ear. “Pilot: Subject Milo.”

Milo. That was what they called him. Not “the monkey.” Not today. Today he had a name, a suit, and a job.

In the control room, behind thick glass and safer walls, Dr. Larkin watched the monitors. His fingers drummed on the console. Every screen showed Milo. Milo’s heartbeat. Milo’s hands. Milo’s pupils, wide and black under the helmet.

Just a few months earlier, this had been a wild idea on a whiteboard.

First, the space agency had a problem: human test pilots were expensive, slow to train, and fragile in the eyes of the public. Crashes broke more than metal. They broke careers, budgets, reputations. But animals? Animals were different. The old reports showed it—dogs, chimps, and mice hurled into the sky for “research.”

Then the aerospace company arrived with a pitch. A guided rocket system. Cheaper launches. Faster data. A trained animal at the controls. They called it “extraordinary innovation.” The lawyers called it “low liability.” The investors called it “brilliant.”

The team called it opportunity.

The university lab, desperate for funding, signed on. Dr. Larkin, still smarting from a failed project, took the lead. His staff followed. Promotions glimmered at the end of every meeting. AURORA was born—a sleek white rocket, narrow as a needle, wired with sensors and cameras.

They picked Milo from a row of caged bodies. He was curious. Fast with puzzles. Quick to touch, quick to learn. A perfect pilot, they said.

They started in a simulator.

First: pictures on a screen. When a green dot appeared, Milo pushed a joystick. Banana mash rewarded him. Beep. Treat. Beep. Treat. He learned quickly.

Then: moving shapes. A small rocket icon drifted across a simple sky. Tilt the stick left, the rocket moved left. Tilt right, it swerved right. Every correct move released food through a tube. Beep. Treat. Beep. Treat.

Later: the seat. The harness. The helmet. They strapped him in while the simulator played. The room darkened. Fans roared. Speakers blasted the sound of engines. They watched him closely as simulated turbulence shook his world. He panicked the first time. He screamed, fought the straps, clawed at the helmet.

They turned the volume down. Gave him more treats. Calmed him. Tried again.

Eventually, he stopped screaming.

He learned the pattern: the rumble meant work, work meant joystick, joystick meant reward. He didn’t know about risk. He didn’t know about people downrange, under the future flight path. He didn’t know that the rocket he was learning to control was real.

On June 5, they cut a corner.

The team had planned ten full simulations. They ran only one complete run-through with the final software. One. The logs showed it. The emails whispered it: “Schedule pressure.” “Launch window.” “We can’t lose the slot.” The company wanted the publicity. A monkey flying a real rocket. Cameras. Headlines. Maybe a viral clip.

Larkin hesitated. Just once. Late at night, staring at the safety report. A red flag blinked at him from the screen: “Autopilot override glitch—requires additional testing.” He clicked past it. The next morning, funding officials were visiting. The rocket had to rise.

The safety systems were supposed to keep everyone safe.

If Milo froze, the autopilot would take over. If systems failed, the rocket would shut down and fall away from the town. That was the promise. The diagrams made it neat and simple. Flow charts. Green arrows. Fail-safes.

“Just like any modern guided rocket,” they told the board. “Only the pilot is an animal, trained like any lab subject. You’ve seen this before. Dogs in orbit. Chimps in capsules. This is just the next step.”

But that step had cracks.

On launch day, the test facility buzzed with nervous stillness. Technicians slid between consoles. Telemetry feeds flickered. Numbers crawled. Outside, the rocket stood on its pad, wrapped in scaffolding and mist from the fueling lines.

In nearby neighborhoods, people went about their day. Children walked to school. Cars idled at lights. No one thought about the thin white cylinder pointing over their roofs.

In the cabin, Milo listened to the countdown echo through the speaker. He didn’t understand the words, but he felt the rhythm. Ten. Nine. Eight.

The vibrations started as a faint tremor under his feet. Loose straps quivered. The joystick trembled in his grip. He smelled something sharp and clean in the air. Fuel. Ozone. Metal warming and groaning.

Seven. Six.

Shadows danced on the panels. A red indicator light glowed on a small display he couldn’t read.

Five. Four.

The hum beneath him became a roar. The cabin shook. His vision blurred as the walls vibrated. His heart pounded, faster than the monitor’s beeping.

Three.

In the control room, an engineer leaned close to Larkin. “We’re still seeing that override flag in the autopilot. It’s probably just a software—”

“Too late,” Larkin said. His voice felt too thin in his throat. “We’re committed.”

Two.

The autopilot light flickered.

One.

The rocket slammed upward.

Sound collapsed into a single, crushing roar. Milo was pushed back into the seat, chest pinned. The sky outside—if he could have seen it—would have been a blur of blue and white. Inside, the world tightened to the joystick, the straps, the shaking.

He gripped.

The training kicked in. The simulator had taught him: when the rocket icon drifted too far, push the stick. Keep it centered. Beep. Treat. Beep. Treat.

Now there was no beep. No treat. Only the scream of the engines and the metallic rattle of every bolt.

The rocket climbed. Telemetry streamed into the control room. Lines of numbers. Velocity. Altitude. Vibration. At first, everything matched the models. Larkin felt his shoulders loosen. “See?” he said, almost to himself. “It’s working. Just like the simulations.”

Then the line on one of the graphs twitched.

A gust of high-altitude wind nudged the rocket off its planned path. Only a little. A degree. A fraction. But the system noticed. The guidance program waited for Milo.

He did as he had been taught.

The rocket veered left.

In the simulator, that move was harmless. In the real sky, that tiny correction pointed the nose toward the edge of the safe zone. Over the boundary. Closer to where people lived.

“Autopilot should catch this,” a technician muttered, eyes locked on the readings.

The autopilot didn’t engage.

The glitch in the override system—flagged, noted, never fixed—hung there, invisible, like a spider in the code. The rocket kept listening to Milo’s hands instead.

He felt the tilt. He pushed again. The craft rolled just a bit more.

Below, in the town, a dog started barking at the distant rumble. Windows vibrated softly. A child looked up, shading his eyes, but the rocket was still too high to see.

Inside the cabin, a warning indicator flashed red. Milo flinched at the sudden brightness. His fingers jerked on the stick. The rocket’s nose dipped.

In the control room, three alarms lit the panel at once.

“Angle deviation!” someone shouted.

“Override!” another called. “Switching to autopilot!”

Fingers flew over keys. A hand slapped a physical toggle. A command shot up the thin thread of radio connection to the rocket.

Nothing happened.

The system believed the pilot still had control. The bug held the door shut. Milo, overwhelmed by noise and pressure, clung harder to the joystick, every twitch translated into tons of thrust and steel.

The rocket dropped a few more degrees toward the town.

“Kill it,” Larkin said. No more hesitation. “Shut it down. Flight termination.”

A safety officer reached for the red switch that would send a final command. Cut the engines. Break the rocket apart in the air. Let the pieces fall short, into the designated empty fields, not the streets.

Outside, clouds drifted. People walked. No one knew they were one command away from disaster.

Static hissed across the speakers as the command went out.

In the cabin, Milo squealed. A harsh crackle filled his ears. The engines surged, then stuttered. The vibrations changed, deepening into an ugly shudder that shook the joystick in his hands.

He didn’t know what a stall felt like. He just knew fear. The seat bit into him as the rocket pitched.

It began to fall.

Not straight. Not clean. The guidance system, confused by the conflicting signals, tried to correct as the engines died. The rocket rolled, nose swinging, painting a wide, invisible curve across the sky.

In the control room, the screens flickered.

Telemetry broke into jagged bursts. Altitude dropping. Angle increasing. Someone swore under their breath. Someone else whispered, “No, no, no—”

A blurred shape streaked across the edge of the town’s security camera footage—a thin white body tumbling through the morning light.

The impact came seconds later.

On the outskirts, near a cluster of low houses, a blast tore through the quiet. Windows shattered. The ground leapt. A shockwave slammed into walls and people and trees. A plume of fire and smoke clawed at the sky, turning blue into gray.

Children screamed. An alarm wailed from a nearby car, shrill and senseless.

In the control room, silence flooded in as the sound from the microphones cut off. Every monitor froze on the same image: a blurred view of instruments just before they went black.

Larkin stared.

He saw not just the smoke on the external feed, not just the broken numbers. He saw the whole chain, stretched out behind him. The funding calls. The emails. The meeting where he’d said, “We’ve launched animals before. This is nothing new.” The nighttime warning on the safety report he’d clicked past.

The idea had seemed clean: extraordinary technology, ordinary responsibility.

But responsibility had slipped.

Someone died in the blast. More were hurt. Houses burned. Families lost what little they had. All because a rocket—piloted by a trained monkey—left the ground without the safeguards it was promised.

Later, investigators would find everything.

The training videos, showing Milo learning to move a simulated rocket for food. The internal emails about schedule pressure. The telemetry logs tracing the sharp turn toward the town. The software reports warning about the autopilot glitch. The safety memo recommending more tests before launch.

Every step, recorded. Every decision, traceable.

They would see the monkey. The rocket. The people below. They would see how the extraordinary was built from ordinary choices, each one a little more careless than the last.

But in the moment, in the echo of the explosion, there was only the smell of burned metal and smoke drifting over broken glass. Only the ringing in ears and the crunch of debris under boots. Only the quiet where a roar had been.

And somewhere at the center of the crater, twisted in the wreckage of straps and wires, lay the pilot they had called Milo.

\n\n--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 98.45 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                             